In [2]:
import tkinter
import cs as cs
import pip
import simpy as simpy


In [3]:
import collections
import random
import simpy

VELOCIDAD_LLEGADA = 42
NUM_BOLETO = 50
TIEMPO_SIMULACION = 120

boletos_vendidos={}
def ventaBoletos(env, num_boletos, pelicula, teatro):
  with teatro.contador.request() as turno:
    resultado = yield turno | teatro.sold_out[pelicula]
    if turno not in resultado:
      teatro.num_renegados[pelicula] += 1
      return
    if teatro.num_boletos[pelicula] < num_boletos:
      yield env.timeout(0.5)
      return
    else:
      if pelicula in list(boletos_vendidos.keys()):
        a=boletos_vendidos.get(pelicula)
        a.append(num_boletos)
        boletos_vendidos[pelicula]=a
      else:
        boletos_vendidos[pelicula]=[num_boletos]

    teatro.num_boletos[pelicula] -= num_boletos
    if teatro.num_boletos[pelicula] < 2:
      teatro.sold_out[pelicula].succeed()
      teatro.tiempo_agotado[pelicula] = env.now
      teatro.num_boletos[pelicula] = 0
    yield env.timeout(1)


def llegadaClientes(env, teatro):
  while True:
    yield env.timeout(random.expovariate(1/0.5))
    pelicula = random.choices(teatro.peliculas, teatro.probabilidad, k=1)
    num_boletos = random.randint(1, 6)
    if teatro.num_boletos[pelicula[0]]:
      env.process(ventaBoletos(env, num_boletos, pelicula[0], teatro))


Teatro = collections.namedtuple('Teatro', 'contador, peliculas, probabilidad, num_boletos, sold_out, tiempo_agotado, num_renegados')

print('Teatro Carlos Crespi - UPS')
env = simpy.Environment()

contador = simpy.Resource(env,capacity=1)
peliculas = ['Conjuro 3', 'Rapidos y Furiosos 10', 'Pulp Fictions']
probabilidad=[0.1, 0.3, 0.6]
num_boletos = {pelicula: NUM_BOLETO for pelicula in peliculas}
sold_out = {pelicula: env.event() for pelicula in peliculas}
tiempo_agotado = {pelicula: None for pelicula in peliculas}
num_renegados = {pelicula: 0 for pelicula in peliculas}

teatro = Teatro(contador, peliculas, probabilidad, num_boletos, sold_out, tiempo_agotado, num_renegados)
env.process(llegadaClientes(env, teatro))
env.run(until=TIEMPO_SIMULACION)

# Analisis y resultados
for pelicula in peliculas:
  if teatro.sold_out[pelicula]:
    print('Pelicula: %s se agoto en el tiempo %.1f despues de salir a la venta' %(pelicula, teatro.tiempo_agotado[pelicula]))
    print('Numero de personas que salieron de la fila/renegados %s' %teatro.num_renegados[pelicula])

total_Pulp=sum(boletos_vendidos['Pulp Fictions'])
total_conjuro=sum(boletos_vendidos['Conjuro 3'])
total_rapidos=sum(boletos_vendidos['Rapidos y Furiosos 10'])

print('Rapidos y furiosos 10:',total_rapidos)
print('Conjuro 3:',total_conjuro)
print('Pulp Fictions:',total_Pulp)

Teatro Carlos Crespi - UPS
Pelicula: Conjuro 3 se agoto en el tiempo 97.2 despues de salir a la venta
Numero de personas que salieron de la fila/renegados 1
Pelicula: Rapidos y Furiosos 10 se agoto en el tiempo 37.3 despues de salir a la venta
Numero de personas que salieron de la fila/renegados 0
Pelicula: Pulp Fictions se agoto en el tiempo 23.3 despues de salir a la venta
Numero de personas que salieron de la fila/renegados 16
Rapidos y furiosos 10: 50
Conjuro 3: 49
Pulp Fictions: 49


**Tarea/Practica**

Al modelo de simulación de la venta de boletos de un teatro/cine agregar la simulación grafica 2D/3D utilizando cualquier libreria grafica, por ejemplo pygame/tkinter, etc. generar un informe en formato PDF que me permita evidenciar la simulación y generar una grafica/reporte de los resultados.



In [4]:
import matplotlib
matplotlib.use('Agg')

**Grafica 2D**

In [5]:
import tkinter as tk
import matplotlib

matplotlib.use('TkAgg')

from matplotlib.figure import Figure
from matplotlib.backends.backend_tkagg import (
    FigureCanvasTkAgg,
    NavigationToolbar2Tk
)

class App(tk.Tk):
    def __init__(self):
        super().__init__()

        self.title('Grafica')

        # # prepare data
        data =num_renegados
        languages = data.keys()
        popularity = data.values()

        # prepare data
        #data1 =boletos_vendidos
        #boletos = data1.keys()
        #vendidos = data1.values()

        # create a figure
        figure = Figure(figsize=(6, 4), dpi=100)

        # create FigureCanvasTkAgg object
        figure_canvas = FigureCanvasTkAgg(figure, self)

        # create the toolbar
        NavigationToolbar2Tk(figure_canvas, self)

        # create axes
        axes = figure.add_subplot()

        # # create the barchart
        axes.bar(languages, popularity)
        axes.set_title('Renegados por pelicula')
        axes.set_ylabel('#')

        # # create the barchart
        # axes.bar(boletos, vendidos)
        # axes.set_title('Boletos vendidos por pelicula')
        # axes.set_ylabel('#')

        figure_canvas.get_tk_widget().pack(side=tk.TOP, fill=tk.BOTH, expand=1)


if __name__ == '__main__':
    app = App()
    app.mainloop()

**Grafica 3D**

In [7]:
import tkinter
import numpy as np
from mpl_toolkits.mplot3d import Axes3D
from matplotlib.backends.backend_tkagg import (
                                    FigureCanvasTkAgg, NavigationToolbar2Tk)
from matplotlib.figure import Figure

root = tkinter.Tk()
root.wm_title("Boletos Vendidos por pelicula")

fig = Figure(figsize=(5, 4), dpi=100)

canvas = FigureCanvasTkAgg(fig, master=root)  # A tk.DrawingArea.
canvas.draw()

list_of_lists = [list(boletos_vendidos[pelicula])]

data = np.array(list_of_lists)
length = data.shape[0]
width = data.shape[1]
x, y = np.meshgrid(np.arange(length), np.arange(width))

ax = fig.add_subplot(1,1,1, projection='3d')
ax.plot_wireframe(x, y, data, alpha=0.8)

toolbar = NavigationToolbar2Tk(canvas, root)
toolbar.update()
canvas.get_tk_widget().pack(side=tkinter.TOP, fill=tkinter.BOTH, expand=1)


tkinter.mainloop()

**Conclusion**

La pelicula que no renego ningun cliente fue la del Conjuro 3, mientras que en rapidos & furiosos y finalmente en la pelicula Pulp Fictions si hubieron personas que renegaron sus boletos.






